# Train a Model

In [1]:
from pyxtal_ff import PyXtal_FF
import os
import urllib.request

C:\Users\fabia\Anaconda3\envs\pyxtal_ff\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
TrainData = "training.json"
TestData = "test.json"

if not os.path.exists(TrainData):
    if not os.path.exists("data"):
        os.mkdir("data")
    os.chdir("data")
    print("Downloading the training and test data")

    url = "https://raw.githubusercontent.com/materialsvirtuallab/mlearn/master/data/Si/"

    urllib.request.urlretrieve(f"{url}/{TrainData}", TrainData)
    urllib.request.urlretrieve(f"{url}/{TestData}", TestData)
    os.chdir("..")

In [3]:
folder = "Si-snap-zbl/"
descriptor = {
    "type": "SNAP",
    "weights": {"Si": 1.0},
    "Rc": 5.0,
    "parameters": {"lmax": 3},
    "base_potential": {"inner": 1.5, "outer": 2.0},  # zbl potential
    "ncpu": 1,
}

In [4]:
model = {
    "system": ["Si"],
    "hiddenlayers": [12, 12],
    "path": folder,
    #'restart': folder + '12-12-checkpoint.pth',
    "optimizer": {"method": "lbfgs"},
    "force_coefficient": 2e-2,
    "stress_coefficient": 2e-3,
    "stress_group": ["Elastic"],
    "alpha": 1e-6,
    "epoch": 50,  # probably more epochs are required for a good model
}

In [5]:
ff = PyXtal_FF(descriptors=descriptor, model=model)
ff.run(
    mode="train",
    TrainData=os.path.join("data", TrainData),
    TestData=os.path.join("data", TestData),
)




               ______       _    _          _         _______ _______ 
              (_____ \     \ \  / /        | |       (_______|_______)
               _____) )   _ \ \/ / |_  ____| |        _____   _____   
              |  ____/ | | | )  (|  _)/ _  | |       |  ___) |  ___)  
              | |    | |_| |/ /\ \ |_( ( | | |_______| |     | |      
              |_|     \__  /_/  \_\___)_||_|_(_______)_|     |_|      
                     (____/      


        A Python package for Machine Learning Interatomic Force Field
         Developed by Zhu's group at University of Nevada Las Vegas
    The source code is available at https://github.com/qzhu2017/FF-project


=========================== version 0.1.9 =============================

Descriptor parameters:
type        : SNAP
Rc          : 5.0
cutoff      : cosine
lmax        : 3
rfac        : 0.99363


Features and precomputed descriptors exist: Si-snap-zbl/Train_db.dat

Features and precomputed descriptors exist: Si-snap-zbl/

C:\Users\fabia\PycharmProjects\PyXtal_FF\pyxtal_ff\models\optimizers\lbfgs.py:264: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\python_arg_parser.cpp:1055.)
  p.data.add_(step_size, update[offset:offset + numel].view_as(p.data))


eng_loss:  11.128424     force_loss:   0.022831   stress_loss:   0.027699  regularization:   0.000009
    Loss:  11.178962     Energy MAE:     4.7087     Force MAE:     0.5513     Stress MAE:     2.8105
eng_loss:   5.059273     force_loss:   0.023757   stress_loss:   0.030391  regularization:   0.000010
    Loss:   5.113431     Energy MAE:     3.1705     Force MAE:     0.5618     Stress MAE:     3.2708

Iteration    2: 
eng_loss:  39.109918     force_loss:   0.022478   stress_loss:   0.029252  regularization:   0.000034
    Loss:  39.161682     Energy MAE:     8.8395     Force MAE:     0.5466     Stress MAE:     2.9647
eng_loss:   0.033963     force_loss:   0.024584   stress_loss:   0.032594  regularization:   0.000013
    Loss:   0.091153     Energy MAE:     0.2521     Force MAE:     0.5720     Stress MAE:     3.4598

Iteration    3: 
eng_loss:   0.029702     force_loss:   0.024428   stress_loss:   0.031981  regularization:   0.000013
    Loss:   0.086124     Energy MAE:     0.2052   

((0.011267301364870539,
  0.01559294740179334,
  0.9974006352005448,
  0.1636385454140876,
  0.2822139283492548,
  0.8946367632525876,
  0.33590067877753516,
  0.5601857027576925,
  0.9895654770333453),
 (0.015192707639328906,
  0.01971211587978288,
  0.9961492288137568,
  0.18270633927413915,
  0.3033409360763685,
  0.8793616574335962,
  0.5036067173023484,
  0.7946996870227262,
  0.9597547442788876))

# Run MD with ASE using this model

In [6]:
from time import time
from ase.build import bulk
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.langevin import Langevin
from ase import units
from pyxtal_ff import PyXtal_FF
from pyxtal_ff.calculator import PyXtalFFCalculator

In [7]:
def printenergy(a, it, t0):
    """Function to print the potential, kinetic and total energy"""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    t_now = time()
    print(
        "Step: %4d [%6.2f]: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  "
        "Etot = %.3feV "
        % (it, t_now - t0, epot, ekin, ekin / (1.5 * units.kB), epot + ekin)
    )
    return t_now

In [8]:
file = "Si-snap-zbl/12-12-checkpoint.pth"

In [9]:
ff = PyXtal_FF(model={"system": ["Si"]}, logo=False)
ff.run(mode="predict", mliap=file)
calc = PyXtalFFCalculator(ff=ff)

si = bulk("Si", "diamond", a=5.659, cubic=True)
si = si * 5
print("MD simulation for ", len(si), " atoms")
si.set_calculator(calc)

MaxwellBoltzmannDistribution(si, 1000 * units.kB)
dyn = Langevin(si, timestep=5 * units.fs, temperature_K=1000, friction=0.02)
# dyn = VelocityVerlet(si, 5*units.fs)  # 2 fs time step.
t0 = time()
for i in range(10):
    dyn.run(steps=1)
    t_now = printenergy(si, i, t0)
    t0 = t_now

MD simulation for  1000  atoms


C:\Users\fabia\Anaconda3\envs\pyxtal_ff\lib\site-packages\ase\md\md.py:48: FutureWarning: Specify the temperature in K using the 'temperature_K' argument
  warnings.warn(FutureWarning(w))


Step:    0 [ 25.22]: Epot = -5.363eV  Ekin = 0.116eV (T=900K)  Etot = -5.247eV 
Step:    1 [ 12.95]: Epot = -5.334eV  Ekin = 0.089eV (T=687K)  Etot = -5.246eV 
Step:    2 [ 12.75]: Epot = -5.302eV  Ekin = 0.058eV (T=452K)  Etot = -5.244eV 
Step:    3 [ 12.89]: Epot = -5.281eV  Ekin = 0.039eV (T=304K)  Etot = -5.242eV 
Step:    4 [ 14.15]: Epot = -5.277eV  Ekin = 0.038eV (T=292K)  Etot = -5.239eV 
Step:    5 [ 13.36]: Epot = -5.286eV  Ekin = 0.047eV (T=366K)  Etot = -5.239eV 
Step:    6 [ 13.22]: Epot = -5.298eV  Ekin = 0.061eV (T=472K)  Etot = -5.237eV 
Step:    7 [ 13.20]: Epot = -5.306eV  Ekin = 0.070eV (T=540K)  Etot = -5.236eV 
Step:    8 [ 14.15]: Epot = -5.304eV  Ekin = 0.070eV (T=539K)  Etot = -5.235eV 
Step:    9 [ 12.96]: Epot = -5.296eV  Ekin = 0.063eV (T=486K)  Etot = -5.233eV 
